**Collections**

In [21]:
from collections import defaultdict
import json

colors = (
    ('Luke', 'Yellow'),
    ('Paul', 'Blue'),
    ('James', 'Green'),
)

# can pass in a sequence of tuples as well as a dict
fav_colors = defaultdict(str, colors)

fav_colors['Luke'] = 'Red'
# no KeyError raised
fav_colors['John'] = 'Yellow'
print(json.dumps(fav_colors))

{"Paul": "Blue", "Luke": "Red", "James": "Green", "John": "Yellow"}


In [20]:
tree = lambda: collections.defaultdict(tree)
some_dict = tree()
some_dict['colours']['favourite'] = "yellow"
# json.dumps returns a JSON string representation of the Python object
print(json.dumps(some_dict))

{"colours": {"favourite": "yellow"}}


In [ ]:
OrderedDict http://book.pythontips.com/en/latest/collections.html

**Copy operations**

Assignment statements in Python do not copy objects, they create bindings between a target and an object. For collections that are mutable or contain mutable items, a copy is sometimes needed so one can change one copy without changing the other.

In [65]:
d = [2, 2, 2]
e = d[:]
d[2] = 4
e

[2, 2, 2]

The difference between shallow and deep copying is only relevant for compound objects (contain other objects, like lists or class instances)

In [66]:
import copy
a = [[2, 2, 2], [2, 2, 2]]
# shallow copy (could use copy.copy). Without the '[:]', 'b' would just point to the object that 'a' references
b = a[:]
c = copy.deepcopy(a)
a[0][2] = 4
b

[[2, 2, 4], [2, 2, 2]]

**Decorators**

In [63]:
def a_new_decorator(a_func):
    def wrapTheFunction():
        print("Before a_func()")
        a_func()
        print("After a_func()")
    return wrapTheFunction

def has_name():
    pass

@a_new_decorator
def decorated():
    pass
print(has_name, decorated)

<function has_name at 0x0000024B17599B70> <function a_new_decorator.<locals>.wrapTheFunction at 0x0000024B175AE6A8>


The decorator overrode the name and docstring of the function. This is needed for debugging etc., so to solve this use functools.wraps. The following is a use-case of a decorator, similar to what might be used to protect a route in Flask where if authorization credentials are not contained within the request object a return redirect statement is executed.

In [64]:
from functools import wraps
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

can_run = False
print(func())

Function is running
Function will not run


**Duck-typing** is a concept that states that the type of the object is a matter of concern only at runtime, so that you don’t need to to explicitly mention the type of object performing an operation on it. E.g. an error is only raised for a function being called with unsupported objects if/when it is encountered.

**Function caching**
http://book.pythontips.com/en/latest/function_caching.html

**Iterables and Iterators**
An iterable is an object capable of returning its members one at a time. This includes all sequences types (e.g. str, list etc.), as well as non-sequence types which includes dict, file objects and objects of any class with an __iter__() method.

An iterator is an object representing a stream of data.  A classic iterator will be defined using a class with __iter__ and __next__ methods. Repeated calls to the iterator’s __next__() method (or passing it to the built-in function next()) return successive items in the stream. When no more data are available a StopIteration exception is raised. 
Ways to create an iterative function include: generator expressions, create a generator function (defined using yield instead of return), or by defining a class with __iter__ and __next__ methods. As iterators have an __iter__ method, they may be used in most places where other iterables are accepted:

In [ ]:
'MAKE A CLASS E.G.'

In [49]:
def counter(low, high):
    n = low
    while n <= high:
        yield n
        n += 1
counts = counter(2,8)
[a for a in counts if a % 2 == 0]

[2, 4, 6, 8]

A container object (e.g. a list) produces a fresh new iterator each time you pass it to the iter() function or use it in a for loop. Attempting this with an iterator will just return the same exhausted iterator object used in the previous iteration pass, making it appear empty:

In [50]:
[a for a in counts if a % 2 == 0]

[]

**Multiprocessing and multithreading**

Concurrency is when two or more tasks can start, run, and complete in overlapping time periods. It doesn't necessarily mean they'll ever both be running at the same instant. Eg. multitasking on a single-core machine. Parallelism is when two or more tasks  run at the same time, e.g. on a multicore processor. Concurrency can be characterized as a property of a program or system and parallelism as the run-time behaviour of executing multiple tasks at the same time. 

If we ran this program on a computer with a single CPU core, the OS would be switching between the two threads, allowing one thread to run at a time. If we ran this program on a computer with a multi-core CPU then we would be able to run the two threads in parallel - side by side at the exact same time

A thread is a sequence of instructions within a process. It can be thought of as a lightweight process. Threads share the same memory space. A process is an instance of a program running in a computer which can contain one or more threads. A process has its independant memory space. 
    
The threading module is used for working with threads in Python. The CPython implementation has a Global Interpreter Lock (GIL) which allows only one thread to be active in the interpreter at once. This means that threads cannot be used for parallel execution of Python code. While parallel CPU computation is not possible, parallel IO operations are possible using threads. This is because performing IO operations releases the GIL. What are threads used for in Python? 

    In GUI applications to keep the UI thread responsive

    IO tasks (network IO or filesystem IO)

Using threads for these tasks improves performance, since in network IO for example, most of the time is spent waiting for a response from the URL. Threads should not be used for CPU bound tasks as this will actually result in worse performance compared to using a single thread.

For parallel execution of tasks use multiprocessing, a package that supports spawning processes using an API similar to the threading module. It side-steps the GIL by using subprocesses instead of threads. The Pool object which offers a convenient means of parallelizing the execution of a function across multiple input values, distributing the input data across processes (data parallelism). The following example demonstrates the common practice of defining such functions in a module so that child processes can successfully import that module:

**Mutable default arguments**

In the following example, the list object has an empty list as a default value. This assignment is carried out only when the function definition is first evaluated. 

In [28]:
def func1(item, lst=[]):
    lst.append(item)
    return lst

def func2 (item, lst=None):
    if lst is None: 
        lst = []
    lst.append(item)
    return lst

A new list is created each time the function is called if a second argument isn’t provided, so that the output is as follows:

In [29]:
print([func1(i) for i in range(3)])

[[0, 1, 2], [0, 1, 2], [0, 1, 2]]


This feature gives speed and memory boosts, as in most cases you will have immutable default arguments and Python can construct them just once, instead of on every function call. Another benefit is simplicity, as it is easier to understand how the expression is evaluated and thereby make debugging easier. 

To avoid this, the mutable objects used as defaults should be replaced by None, and then the arguments tested for None:

In [30]:
print([func2(i) for i in range(3)])

[[0], [1], [2]]


**Else clauses on loops**
Think of the else as 'nobreak':

In [59]:
mylist = [1,2,3]
for i in mylist:
    print(i)
    if i == 3:
        break
else:
    print('Reached StopIteration or broke out of loop')

1
2
3


**functools.partial**
https://stackoverflow.com/questions/15331726/how-does-the-functools-partial-work-in-python
https://www.pydanny.com/python-partials-are-fun.html

**Factory Methods**

**Property Decorator**  getter setter deleter methods https://docs.python.org/2.7/reference/datamodel.html#customizing-attribute-access
Enable you to set an attribute using a function. The setter method allows you to have functionality of normal class attribute assignment.

-overall useful in that can come in and modify property already defined when the class instance was initalized?

see corey s youtube vid

**Strong-typing** You cannot coerce objects into a different type by any kind of inferring, as in weakly-typed languages. In so, the object type is more explicit.



**sys**

('3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]',
 ['',
  'C:\\Users\\Paul\\Anaconda3\\python35.zip',
  'C:\\Users\\Paul\\Anaconda3\\DLLs',
  'C:\\Users\\Paul\\Anaconda3\\lib',
  'C:\\Users\\Paul\\Anaconda3',
  'C:\\Users\\Paul\\AppData\\Roaming\\Python\\Python35\\site-packages',
  'C:\\Users\\Paul\\Anaconda3\\lib\\site-packages',
  'C:\\Users\\Paul\\Anaconda3\\lib\\site-packages\\win32',
  'C:\\Users\\Paul\\Anaconda3\\lib\\site-packages\\win32\\lib',
  'C:\\Users\\Paul\\Anaconda3\\lib\\site-packages\\Pythonwin',
  'C:\\Users\\Paul\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
  'C:\\Users\\Paul\\.ipython'])